In [3]:
#set up environment
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('data_processing').getOrCreate()
import pyspark.sql.functions as F
from pyspark.sql.types import *


In [10]:
#首先建立一个SparkSession Object，然后建立DataFra me，包含integer和string等5个字段
schema = StructType().add("user_id","string").add("country","string").add("browser","string").add("OS","string").add("age","string")

df = spark.createDataFrame([("A203",'India',"Chrome","WIN",33),("A201",'China',"Safari","MacOS",35),("A205",'UK',"Mozilla","Linux",25)],schema=schema)

In [13]:
#我们可以看一下df的数据结构和数据结构：
df.printSchema()
df.show()

root
 |-- user_id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- browser: string (nullable = true)
 |-- OS: string (nullable = true)
 |-- age: string (nullable = true)

+-------+-------+-------+-----+---+
|user_id|country|browser|   OS|age|
+-------+-------+-------+-----+---+
|   A203|  India| Chrome|  WIN| 33|
|   A201|  China| Safari|MacOS| 35|
|   A205|     UK|Mozilla|Linux| 25|
+-------+-------+-------+-----+---+



In [18]:
缺失值替换 删除column
df_na = spark.createDataFrame([("A203",None,"Chrome","WIN",33)],schema = schema)
df_na.show()
df_na.fillna('0').show()
df_na.fillna({'country':'USA'}).show()
df_na.drop('user_id').show()

+-------+-------+-------+---+---+
|user_id|country|browser| OS|age|
+-------+-------+-------+---+---+
|   A203|   null| Chrome|WIN| 33|
+-------+-------+-------+---+---+

+-------+-------+-------+---+---+
|user_id|country|browser| OS|age|
+-------+-------+-------+---+---+
|   A203|      0| Chrome|WIN| 33|
+-------+-------+-------+---+---+

+-------+-------+-------+---+---+
|user_id|country|browser| OS|age|
+-------+-------+-------+---+---+
|   A203|    USA| Chrome|WIN| 33|
+-------+-------+-------+---+---+

+-------+-------+---+---+
|country|browser| OS|age|
+-------+-------+---+---+
|   null| Chrome|WIN| 33|
+-------+-------+---+---+



In [22]:
#读取数据
df = spark.read.csv("BankChurners.csv",header=True,inferSchema=True)
print(df.count())
print(len(df.columns))
df.printSchema()

10127
21
root
 |-- CLIENTNUM: integer (nullable = true)
 |-- Attrition_Flag: string (nullable = true)
 |-- Customer_Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Dependent_count: integer (nullable = true)
 |-- Education_Level: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Income_Category: string (nullable = true)
 |-- Card_Category: string (nullable = true)
 |-- Months_on_book: integer (nullable = true)
 |-- Total_Relationship_Count: integer (nullable = true)
 |-- Months_Inactive_12_mon: integer (nullable = true)
 |-- Contacts_Count_12_mon: integer (nullable = true)
 |-- Credit_Limit: double (nullable = true)
 |-- Total_Revolving_Bal: integer (nullable = true)
 |-- Avg_Open_To_Buy: double (nullable = true)
 |-- Total_Amt_Chng_Q4_Q1: double (nullable = true)
 |-- Total_Trans_Amt: integer (nullable = true)
 |-- Total_Trans_Ct: integer (nullable = true)
 |-- Total_Ct_Chng_Q4_Q1: double (nullable = true)
 |-- Avg_Utilization_Ratio: 

In [24]:
df.select(['Card_Category','Credit_Limit']).show()


+-------------+------------+
|Card_Category|Credit_Limit|
+-------------+------------+
|         Blue|     12691.0|
|         Blue|      8256.0|
|         Blue|      3418.0|
|         Blue|      3313.0|
|         Blue|      4716.0|
|         Blue|      4010.0|
|         Gold|     34516.0|
|       Silver|     29081.0|
|         Blue|     22352.0|
|         Blue|     11656.0|
|         Blue|      6748.0|
|         Blue|      9095.0|
|         Blue|     11751.0|
|         Blue|      8547.0|
|         Blue|      2436.0|
|         Blue|      4234.0|
|         Blue|     30367.0|
|         Blue|     13535.0|
|         Blue|      3193.0|
|         Blue|     14470.0|
+-------------+------------+
only showing top 20 rows



In [29]:
print(df.filter(df['Credit_Limit']>30000).count())
df.filter(df['Credit_Limit']>30000).show()

667
+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+-----

In [31]:
df.filter(df['Credit_Limit']>30000).filter(df['Card_Category']=='Gold').show()

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------

In [32]:
df.where((df['Credit_Limit']>30000) & (df['Card_Category']=='Gold')).show()

+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+
|CLIENTNUM|   Attrition_Flag|Customer_Age|Gender|Dependent_count|Education_Level|Marital_Status|Income_Category|Card_Category|Months_on_book|Total_Relationship_Count|Months_Inactive_12_mon|Contacts_Count_12_mon|Credit_Limit|Total_Revolving_Bal|Avg_Open_To_Buy|Total_Amt_Chng_Q4_Q1|Total_Trans_Amt|Total_Trans_Ct|Total_Ct_Chng_Q4_Q1|Avg_Utilization_Ratio|
+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------